In [105]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import unicodedata

import re
from openpyxl import Workbook, load_workbook

In [98]:
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [3]:
# 엑셀 생성
wb = Workbook()
ws = wb.active

# 제목 적기
sub = ['식별키','목록-번호','목록-구분','목록-제목','목록-면수','목록-작성자','목록-일자','본문-일자(원본)','본문-일자','본문-내용','본문-제목','본문-부제(제목앞)','본문-부제(제목뒤)']
for kwd, j in zip(sub, list(range(1, len(sub)+1))):
    ws.cell(row=1, column=j).value = kwd

wb.save('./output/노동신문.xlsx')

In [99]:
list_base_url = 'http://www.rodong.rep.kp/ko/index.php?strPageID=SF01_01_03&strDate='
date = '2022-07-01'

In [100]:
browser.get(list_base_url+date)

### 크롤링 범위
- 목록-번호 / 목록-구분 / 목록-제목 / 목록-면 / 목록-작성자 / 목록-일자
- 본문-일자(원본) / 본문-일자 / 본문-내용 / 본문-제목 / 본문-부제(제목앞) / 본문-부제(제목뒤)

- '식별키','목록-번호','목록-구분','목록-제목','목록-면수','목록-작성자','목록-일자','본문-일자(원본)','본문-일자','본문-내용','본문-제목','본문-부제(제목앞)','본문-부제(제목뒤)'

## 뉴스 목록 크롤링

In [6]:
news_line_list = browser.find_elements(by=By.CLASS_NAME, value='ListNewsLineContainer')

In [7]:
def get_article_info(title_full) :
    if title_full.startswith('[') :
        article_title = title_full[title_full.find(']')+1 : title_full.rfind('[')].strip()
        article_type = title_full[title_full.find('[')+1 : title_full.find(']')].strip()
        article_page = title_full[title_full.rfind('[')+1 : title_full.rfind(']')].strip()
    else :
        article_title = title_full[ : title_full.rfind('[')].strip()
        article_type = ''
        article_page = title_full[title_full.rfind('[')+1 : title_full.rfind(']')].strip() 
    
    return article_title, article_type, article_page

In [67]:
for news_line in news_line_list:
    temp_row = []
    
    line_no = news_line.find_element(by=By.CLASS_NAME, value='ListNewsLineNo').text.strip()
    line_title = news_line.find_element(by=By.CLASS_NAME, value='ListNewsLineTitleW').text.strip()
    line_writer = news_line.find_element(by=By.CLASS_NAME, value='ListNewsLineWriter').text.strip()
    line_date = news_line.find_element(by=By.CLASS_NAME, value='ListNewsLineDate').text.strip()
    
    article_title, article_type, article_page = get_article_info(line_title)

    temp_row.append(line_no)
    temp_row.append(article_type)
    temp_row.append(article_title)
    temp_row.append(article_page)
    temp_row.append(line_writer)
    temp_row.append(line_date)
    
    # 상세창 팝업
    title_popup = news_line.find_element(by=By.CLASS_NAME, value='ListNewsLineTitleW')
    title_popup_a = title_popup.find_element(by=By.TAG_NAME, value='a')
    title_popup_a.click()    
    browser.switch_to.window(browser.window_handles[1])
    
    key = browser.current_url[-15:]
    temp_row.insert(0, key)
    
    detail_date_origin = browser.find_element(by=By.CLASS_NAME, value='ArticleMenuDate').text
    temp_row.append(detail_date_origin)
    
    detail_date = key[:10]
    temp_row.append(detail_date)
    
    detail_title = ''
    detail_sub_title = ''
    detail_content = ''
    
    detail_content_list = browser.find_elements(by=By.CLASS_NAME, value='ArticleContent')

    for detail_line in detail_content_list:
        title = detail_line.find_elements(by=By.TAG_NAME, value='font')

        if title and title[0].text.strip() != '' :
            if 'bold' in title[0].get_attribute('style') :
                detail_title += title[0].text + ' '
            else :
                detail_sub_title += title[0].text + ' '
        elif detail_line.text.strip() != '' :
            detail_content += ' ' + detail_line.text
    
    temp_row.append(detail_content)
    temp_row.append(detail_title.strip())
    temp_row.append(detail_sub_title.strip())
    temp_row.append(detail_sub_title.strip())
    ws.append(temp_row)
    wb.save('./output/노동신문.xlsx')
    
    # 팝업창 close 후 객체 전환
    browser.close() 
    browser.switch_to.window(browser.window_handles[0])

In [16]:
title_popup = news_line.find_element(by=By.CLASS_NAME, value='ListNewsLineTitleW')
title_popup_a = title_popup.find_element(by=By.TAG_NAME, value='a')
title_popup_a.click()

In [51]:
browser.window_handles

['CDwindow-2FACECDB0A3970DC7739F128E93F439C',
 'CDwindow-6148C768679F565202DC9F8ABABF7B45',
 'CDwindow-13C5389092528A7700BC5A53049A293B']

In [8]:
browser.switch_to.window(browser.window_handles[-1])

In [9]:
key = browser.current_url[-15:]
key

'2022-06-01-0008'

## 뉴스 상세 크롤링

In [5]:
detail_date_origin = browser.find_element(by=By.CLASS_NAME, value='ArticleMenuDate').text
detail_date_origin

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".ArticleMenuDate"}
  (Session info: chrome=103.0.5060.134)
Stacktrace:
0   chromedriver                        0x00000001045f9d60 chromedriver + 3792224
1   chromedriver                        0x0000000104590874 chromedriver + 3360884
2   chromedriver                        0x0000000104291184 chromedriver + 217476
3   chromedriver                        0x00000001042c0f5c chromedriver + 413532
4   chromedriver                        0x00000001042e9b2c chromedriver + 580396
5   chromedriver                        0x00000001042b63e4 chromedriver + 369636
6   chromedriver                        0x00000001045cf234 chromedriver + 3617332
7   chromedriver                        0x00000001045d36c4 chromedriver + 3634884
8   chromedriver                        0x00000001045d7cb8 chromedriver + 3652792
9   chromedriver                        0x00000001045d415c chromedriver + 3637596


In [11]:
detail_date = key[:10]
detail_date

'2022-06-01'

In [102]:
detail_title = ''
detail_sub_title_pre = ''
detail_sub_title_next = ''
detail_content = ''

In [103]:
def get_text_type(line, detail_title, detail_content):
    font_tag = line.find_elements(by=By.TAG_NAME, value='font')
    if len(font_tag) > 0 and 'font-size' in font_tag[0].get_attribute('style'):
        if 'bold' in font_tag[0].get_attribute('style') : # bold인 것들 주제목
            return "TITLE"
        elif len(detail_title) > 0 and len(detail_content) == 0: # 제목은 나오고, 내용은 나오지 않은 경우
            return "POST_TITLE"
        elif len(detail_title) == 0 and len(detail_content) == 0: # 제목과 내용이 아직 나오지 않은 경우
            return "PRE_TITLE"
        else:
            return "CONTENT"
    else:
        return "CONTENT"

In [104]:
detail_content_list = browser.find_elements(by=By.CLASS_NAME, value='ArticleContent')

for detail_line in detail_content_list:
    line_type = get_text_type(detail_line, detail_title, detail_content)
    print(line_type, detail_line.text)
    if line_type == "TITLE":
        detail_title += detail_line.text + '\r\n'
    elif line_type == "PRE_TITLE":
        detail_sub_title_pre += detail_line.text + '\r\n' # 부제목(앞)
    elif line_type == "POST_TITLE":
        detail_sub_title_next += detail_line.text + '\r\n' # 부제목(뒤)
    elif line_type == "CONTENT":
        detail_content += detail_line.text + '\r\n' # 본문 내용
#     title = detail_line.find_elements(by=By.TAG_NAME, value='font')
    
#     if title and title[0].text.strip() != '' :
# #         print(title[0].text)
#         if 'bold' in title[0].get_attribute('style') : # bold인 것들 주제목
#             detail_title += title[0].text + '\r\n'
#         elif len(detail_title) > 0 and len(detail_content) == 0: # 제목이 이미 들어가고, 내용은 나오지 않은 경우
#             detail_sub_title_next += title[0].text + '\r\n' # 부제목(뒤)
#         elif len(detail_content) == 0:
#             detail_sub_title_pre += title[0].text + '\r\n' # 부제목(앞)
#     elif detail_line.text.strip() != '' and 'right' not in detail_line.get_attribute('style') : # 우측정렬은 글, 사진 작성자
#         detail_content += detail_line.text + '\r\n'

TITLE 위대한 년대의 불멸할 자욱을 길이 전하는 고귀한 재보
POST_TITLE 지난 １０년간 조국해방전쟁승리기념관에 많은 자료와 유물, 사진들이 보충전시되였다
CONTENT  
CONTENT 조국해방전쟁승리기념관은 위대한 수령 김일성동지의 불멸의 전승업적을 만대에 길이 전하는 승리전통교양의 중심지, 민족의 대국보이다.
CONTENT 조국해방전쟁승리기념관을 세계적인 기념관으로 훌륭히 꾸리려는 우리 당의 웅대한 구상과 정력적인 령도에 의해 주체１０１(２０１２)년 ９월 건설이 시작된 때로부터 지난 １０년간 기념관에는 많은 사적자료와 전시유물, 사진들이 새로 발굴전시되였다.
CONTENT 경애하는 김정은동지께서는 다음과 같이 말씀하시였다.
CONTENT 《조국해방전쟁에서의 승리는 수령님의 탁월한 군사사상과 주체전법, 령활한 전략전술의 승리이며 수령님의 부르심을 받들고 조국을 지키기 위하여 목숨도 서슴없이 바쳐싸운 우리 군대와 인민의 불타는 애국심과 백절불굴의 투쟁정신의 승리입니다.》
CONTENT 지금으로부터 １０년전 이곳을 찾으신 경애하는 총비서동지께서는 조국해방전쟁승리기념관은 인민군군인들과 근로자들, 청년학생들을 위대한 수령님의 주체사상과 반제혁명사상, 탁월한 령군술과 군사전략전술 및 전법으로 무장시키는 훌륭한 교양거점이라고 하시면서 조국해방전쟁과 관련한 사적물들과 전시물들을 더 발굴하기 위한 사업을 심도있게 벌릴데 대하여 가르치시였다.
CONTENT 경애하는 총비서동지의 숭고한 뜻을 높이 받들고 조국해방전쟁승리기념관 일군들과 학술연구원, 전쟁로병들과 그 유가족들, 각지 당원들과 근로자들은 전시유물, 유품발굴사업에 적극 참가하였다.
CONTENT 조국해방전쟁승리기념관건설이 시작된 때로부터 개관할 때까지 근 １년간에만도 ９０여종에 １ ２００여점의 전시유물, 유품들이 기증되였다.
CONTENT 그중에는 항일혁명투사 최현동지가 전쟁시기 용감하게 싸운 한 군인에게 그 무엇에도 비길수 없는 귀중한 재보라고 하면서 안겨준 위대한 수령님의 사진문헌과 우리 수령님의 친필존

In [109]:
print(detail_sub_title_pre)
print(unicodedata.normalize('NFKC', detail_title))
print(unicodedata.normalize('NFKC', detail_sub_title_next))
print(unicodedata.normalize('NFKC', detail_content))


위대한 년대의 불멸할 자욱을 길이 전하는 고귀한 재보

지난 10년간 조국해방전쟁승리기념관에 많은 자료와 유물, 사진들이 보충전시되였다

 
조국해방전쟁승리기념관은 위대한 수령 김일성동지의 불멸의 전승업적을 만대에 길이 전하는 승리전통교양의 중심지, 민족의 대국보이다.
조국해방전쟁승리기념관을 세계적인 기념관으로 훌륭히 꾸리려는 우리 당의 웅대한 구상과 정력적인 령도에 의해 주체101(2012)년 9월 건설이 시작된 때로부터 지난 10년간 기념관에는 많은 사적자료와 전시유물, 사진들이 새로 발굴전시되였다.
경애하는 김정은동지께서는 다음과 같이 말씀하시였다.
《조국해방전쟁에서의 승리는 수령님의 탁월한 군사사상과 주체전법, 령활한 전략전술의 승리이며 수령님의 부르심을 받들고 조국을 지키기 위하여 목숨도 서슴없이 바쳐싸운 우리 군대와 인민의 불타는 애국심과 백절불굴의 투쟁정신의 승리입니다.》
지금으로부터 10년전 이곳을 찾으신 경애하는 총비서동지께서는 조국해방전쟁승리기념관은 인민군군인들과 근로자들, 청년학생들을 위대한 수령님의 주체사상과 반제혁명사상, 탁월한 령군술과 군사전략전술 및 전법으로 무장시키는 훌륭한 교양거점이라고 하시면서 조국해방전쟁과 관련한 사적물들과 전시물들을 더 발굴하기 위한 사업을 심도있게 벌릴데 대하여 가르치시였다.
경애하는 총비서동지의 숭고한 뜻을 높이 받들고 조국해방전쟁승리기념관 일군들과 학술연구원, 전쟁로병들과 그 유가족들, 각지 당원들과 근로자들은 전시유물, 유품발굴사업에 적극 참가하였다.
조국해방전쟁승리기념관건설이 시작된 때로부터 개관할 때까지 근 1년간에만도 90여종에 1 200여점의 전시유물, 유품들이 기증되였다.
그중에는 항일혁명투사 최현동지가 전쟁시기 용감하게 싸운 한 군인에게 그 무엇에도 비길수 없는 귀중한 재보라고 하면서 안겨준 위대한 수령님의 사진문헌과 우리 수령님의 친필존함이 모셔진 군공메달증서, 수령님께서 한 전사에게 주신 라지오와 사진확대기도 있다.어버이수령님의 존귀하신 영상이 모셔진 상장과 싸우는 고지의 병사들과 후방

In [101]:
# 팝업창 종료 후 close
# browser.close() 
browser.switch_to.window(browser.window_handles[-1])